# 📊 Market Intelligence Analysis
This notebook analyzes global financial assets using Yahoo Finance data.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import plotly.express as px
sns.set(style='whitegrid')

In [ ]:
# Define assets
assets = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'META', 'GLD', 'BTC-USD', 'EURUSD=X', '^GSPC']
df = yf.download(assets, start='2020-01-01', end='2024-12-31')['Adj Close']
df.to_csv('../data/downloaded_data.csv')
df.head()

## 📈 Returns & Risk

In [ ]:
returns = df.pct_change().dropna()
annual_return = returns.mean() * 252
annual_volatility = returns.std() * np.sqrt(252)
sharpe_ratio = annual_return / annual_volatility
risk_return = pd.DataFrame({'Return': annual_return, 'Volatility': annual_volatility, 'Sharpe': sharpe_ratio})
risk_return.plot(kind='scatter', x='Volatility', y='Return', figsize=(10,6), s=100, alpha=0.7)
for i in risk_return.index:
    plt.annotate(i, (risk_return.loc[i,'Volatility'], risk_return.loc[i,'Return']))
plt.title('Risk vs Return')
plt.savefig('../graphs/risk_return.png')
plt.show()

## 🔥 Correlation Heatmap

In [ ]:
correlation = returns.corr()
plt.figure(figsize=(10,8))
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.savefig('../graphs/correlation_heatmap.png')
plt.show()

## 🔍 PCA & Clustering

In [ ]:
pca = PCA(n_components=2)
components = pca.fit_transform(returns.T)
pca_df = pd.DataFrame(components, columns=['PC1', 'PC2'], index=returns.columns)
kmeans = KMeans(n_clusters=3, n_init=10).fit(pca_df)
pca_df['Cluster'] = kmeans.labels_
fig = px.scatter(pca_df, x='PC1', y='PC2', color='Cluster', text=pca_df.index, title='PCA Clustering')
fig.write_html('../graphs/pca_clustering.html')
fig.show()